In [ ]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
import joblib
import plotly.graph_objects as go

source_path = "../Source"
if source_path not in sys.path:
    sys.path.append(source_path)
from baseline_preprocessing import Cleaner, Encoder, Scaler

In [11]:
csv_path = r"C:\Users\Rasulbek907\Desktop\Hotel Booking Cancellation Prediction\Data\Raw_Data\hotel_bookings_updated_2024.csv"
df = pd.read_csv(csv_path)
y = df['is_canceled']
X = df.drop(columns=['is_canceled'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
cleaner = Cleaner(); cleaner.fit(X_train)
X_train_clean = cleaner.transform(X_train)
X_test_clean = cleaner.transform(X_test)

encoder = Encoder(max_unique=5); encoder.fit(X_train_clean)
X_train_enc = encoder.transform(X_train_clean)
X_test_enc = encoder.transform(X_test_clean)

scaler = Scaler(); scaler.fit(X_train_enc)
X_train_final = scaler.transform(X_train_enc)
X_test_final = scaler.transform(X_test_enc)

In [13]:
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_final, y_train)

In [14]:
lr = LogisticRegression(max_iter=1000)
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
models = {"Logistic Regression": lr, "Decision Tree": dt, "Random Forest": rf, "KNN": knn}

In [15]:
results = []
for name, model in models.items():
    model.fit(X_train_bal, y_train_bal)
    y_pred = model.predict(X_test_final)
    results.append({
        "Model": name,
        "Accuracy": round(accuracy_score(y_test, y_pred),4),
        "Precision": round(precision_score(y_test, y_pred),4),
        "Recall": round(recall_score(y_test, y_pred),4),
        "F1-Score": round(f1_score(y_test, y_pred),4)
    })

results_df = pd.DataFrame(results)
results_df

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,1.000,1.000,1.0000,1.0000
1,Decision Tree,1.000,1.000,1.0000,1.0000
2,Random Forest,1.000,1.000,1.0000,1.0000
3,KNN,0.581,0.413,0.3115,0.3551


In [18]:
best_idx = results_df['F1-Score'].idxmax()
best_model_name = results_df.loc[best_idx, 'Model']
best_model = models[best_model_name]

save_dir = r"C:\Users\Rasulbek907\Desktop\Hotel Booking Cancellation Prediction\Models"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, f"{best_model_name.replace(' ','_')}baseline_model.pkl")
joblib.dump(best_model, save_path)
best_model_name, save_path


('Logistic Regression',
 'C:\\Users\\Rasulbek907\\Desktop\\Hotel Booking Cancellation Prediction\\Models\\Logistic_Regressionbaseline_model.pkl')

In [17]:
colors = []
for i, row in results_df.iterrows():
    row_colors = []
    for metric in ['Accuracy','Precision','Recall','F1-Score']:
        if row[metric] >= 0.8:
            row_colors.append('lightgreen')
        elif row[metric] < 0.6:
            row_colors.append('lightcoral')
        else:
            row_colors.append('white')
    colors.append(['white'] + row_colors)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(results_df.columns), fill_color='paleturquoise', align='center'),
    cells=dict(values=[results_df[col] for col in results_df.columns], fill_color=colors, align='center'))
])
fig.show()